In [34]:
import pandas as pd
import html5lib
import requests
from bs4 import BeautifulSoup
my_url = "https://www.transfermarkt.com/bsc-young-boys/kader/verein/452/saison_id/2019"

In [35]:
# here we define the headers for the request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

# this request object will integrate your URL and the headers defined above
req = urllib.request.Request(url=my_url, headers=headers)

# calling urlopen this way will automatically handle closing the request
with urllib.request.urlopen(req) as response:
    page_html = response.read()

In [38]:
#page = requests.get(my_url)
soup = BeautifulSoup(page_html, 'html.parser')

In [175]:
table = soup.find_all('table',{"class":"items"})
table = table[0]
player_row = table.find_all("tr")

for player_row in player_row[1:]:
    #parse inline_table
    player_tags = player_row.find_all("td")
    player_number = player_tags[0].text
    player_name = player_tags[1].div.text
    player_link = player_tags[1].div.span.a['href']
    player_id = player_tags[1].div.span.a['href']
    
    print(player_tags[1].div.span.a)
    #for player_tag in player_tags:
    #    print(player_tag[])
        
        
    break 

<a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">David von Ballmoos</a>


In [124]:
player_row[:][1].find_all("td")

[<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td>,
 <td class="posrela" title=""><table class="inline-table" title=""><tr><td class="" rowspan="2"><a href="#"><img alt="David von Ballmoos" class="bilderrahmen-fixed" src="https://tmssl.akamaized.net//images/portrait/small/203124-1533134983.jpg?lm=1533134993" title="David von Ballmoos"/></a></td><td class="hauptlink"><div class="di nowrap"><span class="hide-for-small"><a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">David von Ballmoos</a></span></div><div class="di nowrap"><span class="show-for-small"><a class="spielprofil_tooltip" href="/david-von-ballmoos/profil/spieler/203124" id="203124" title="David von Ballmoos">D. von Ballmoos</a></span></div></td></tr><tr><td>Goalkeeper</td></tr></table></td>,
 <td class="" rowspan="2"><a href="#"><img alt="David von Ballmoos" class="bilderrahmen-fixed" src="https://tmssl.ak

In [110]:
player_row[0]

[<th class="zentriert" id="yw1_c0"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">#</a></th>,
 <th id="yw1_c1"><a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">Player</a></th>,
 <th class="zentriert" id="yw1_c2"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">Date of birth / Age</a></th>,
 <th class="zentriert" id="yw1_c3">Nat.</th>,
 <th class="zentriert" id="yw1_c4"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">Contract expires</a></th>,
 <th class="rechts" id="yw1_c5"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">Market value</a></th>]

In [105]:
player_row

<tr>
<th class="zentriert" id="yw1_c0"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">#</a></th><th id="yw1_c1"><a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">Player</a></th><th class="zentriert" id="yw1_c2"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">Date of birth / Age</a></th><th class="zentriert" id="yw1_c3">Nat.</th><th class="zentriert" id="yw1_c4"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">Contract expires</a></th><th class="rechts" id="yw1_c5"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">Market value</a></th></tr>

In [95]:
players[0]

'<tr>\n <th class="zentriert" id="yw1_c0">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">\n   #\n  </a>\n </th>\n <th id="yw1_c1">\n  <a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">\n   Player\n  </a>\n </th>\n <th class="zentriert" id="yw1_c2">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">\n   Date of birth / Age\n  </a>\n </th>\n <th class="zentriert" id="yw1_c3">\n  Nat.\n </th>\n <th class="zentriert" id="yw1_c4">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">\n   Contract expires\n  </a>\n </th>\n <th class="rechts" id="yw1_c5">\n  <a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">\n   Market value\n  </a>\n </th>\n</tr>\n'

In [90]:
players[1].find("title")

In [ ]:
pla

In [82]:
len(table.find_all("tr"))

76

In [ ]:
rows = soup.find("table", border=1).find("tbody").find_all("tr")

for row in rows:
    cells = row.find_all("td")
    rn = cells[0].get_text()

In [79]:
print(table[2])

IndexError: list index out of range

In [72]:
for player in table[0]:
    print(player)



<thead>
<tr>
<th class="zentriert" id="yw1_c0"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">#</a></th><th id="yw1_c1"><a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">Player</a></th><th class="zentriert" id="yw1_c2"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">Date of birth / Age</a></th><th class="zentriert" id="yw1_c3">Nat.</th><th class="zentriert" id="yw1_c4"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">Contract expires</a></th><th class="rechts" id="yw1_c5"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">Market value</a></th></tr>
</thead>


<tbody>
<tr class="odd">
<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td><td class="posrela" title=""><table class="inline-table" title=""><tr><td class="" rowspan="2"

In [70]:
len(tables)

1

In [54]:
print(tables[1])

<table class="items">
<thead>
<tr>
<th class="zentriert" id="yw1_c0"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/rn">#</a></th><th id="yw1_c1"><a class="sort-link asc" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/spieler.desc">Player</a></th><th class="zentriert" id="yw1_c2"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/age">Date of birth / Age</a></th><th class="zentriert" id="yw1_c3">Nat.</th><th class="zentriert" id="yw1_c4"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/vertrag_bis">Contract expires</a></th><th class="rechts" id="yw1_c5"><a class="sort-link" href="/bsc-young-boys/kader/verein/452/saison_id/2019/sort/marktwert.desc">Market value</a></th></tr>
</thead>
<tbody>
<tr class="odd">
<td class="zentriert rueckennummer bg_Torwart" title="Goalkeeper"><div class="rn_nummer">26</div></td><td class="posrela" title=""><table class="inline-table" title=""><tr><td cl

In [50]:
for table in tables:
    print(table)



<tbody>
<tr>
<td>
                                                    Filter by season                                                </td>
<td>
<div class="inline-select">
<select class="chzn-select" data-placeholder="Filter by season" name="saison_id" tabindex="0">
<option selected="selected" value="2019">19/20</option>
<option value="2018">18/19</option>
<option value="2017">17/18</option>
<option value="2016">16/17</option>
<option value="2015">15/16</option>
<option value="2014">14/15</option>
<option value="2013">13/14</option>
<option value="2012">12/13</option>
<option value="2011">11/12</option>
<option value="2010">10/11</option>
<option value="2009">09/10</option>
<option value="2008">08/09</option>
<option value="2007">07/08</option>
<option value="2006">06/07</option>
<option value="2005">05/06</option>
<option value="2004">04/05</option>
<option value="2003">03/04</option>
<option value="2002">02/03</option>
<option value="2001">01/02</option>
<option value="2000">00/01

In [ ]:
#player_table = soup.find_all('responsive-table', class_='outer-text')

In [20]:
table = soup.find('table', attrs={'class':'lineItemsTable'})
#table_body = table.find('tbody')

In [29]:
print(soup.find_all('table'))

[]


In [33]:
soup

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr/><center>nginx</center>
</body>
</html>

In [3]:
import urllib.request  # we are going to need to generate a Request object
from bs4 import BeautifulSoup as soup


# here we define the headers for the request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

# this request object will integrate your URL and the headers defined above
req = urllib.request.Request(url=my_url, headers=headers)

# calling urlopen this way will automatically handle closing the request
with urllib.request.urlopen(req) as response:
    page_html = response.read()

In [5]:
df = pd.read_html(page_html)

In [11]:
df[3]

,0,1
0,NaN,Marco WölfliM. Wölfli
1,NaN,Goalkeeper
